In [2]:
#output csv with content
from urllib import request
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import time

def createDataFrame(t,u):
    #put data into a list
    title_list = [p.text.strip() for p in t]
    url_list = ["https://tfc-taiwan.org.tw" + url.get('href') for url in u]
    
    global df
    #check if dataframe exist
    if c == 0:
        df = pd.DataFrame({'title': title_list, 'url': url_list})
    else:
        df = df.append(pd.DataFrame({'title': title_list, 'url': url_list}), ignore_index = True)
    
    #check fake
    cond1 = df['title'].str.contains('【錯誤】')
    cond2 = df['title'].str.contains('【部分錯誤】')
    '''
    if title contains 【錯誤】:
        put 'fake'
    elif contains 【部分錯誤】:
        put 'partial error'
    else:
        put 'other'
    '''
    df['tag'] = np.where(cond1, '__label__fake', np.where(cond2, '__label__partialError', '__label__other'))
    df = df[['tag','title','url']]
    

def outputCsv(): 
    #input news article URL
    #健康https://tfc-taiwan.org.tw/taxonomy/term/475
    newsURL = "https://tfc-taiwan.org.tw/taxonomy/term/475"
    global c
    c = 0
    for i in range(1):
        req = request.Request(newsURL)
        req.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.122 Safari/537.36")
        response = request.urlopen(req)
        data = response.read().decode("utf-8")
        
        #read data in html format
        html = BeautifulSoup(data,"html.parser")
        
        #set find area
        t = html.find_all("h3", class_= "article-title")
        u = html.select("h3.article-title a")
        
        #create dataframe
        createDataFrame(t,u) 
        
        next_link = html.select("div.item-list a")
        next_url = "https://tfc-taiwan.org.tw/taxonomy/term/475" + next_link[3]["href"]
        newsURL = next_url
        
        c+=1
        time.sleep(3)
    
    df['title'] = df['title'].str.replace('【錯誤】','').str.replace('【部分錯誤】','').str.replace('網傳','')
    pd.set_option('max_colwidth',100)    
    display(df)
    
outputCsv()

tag  \
0          __label__fake   
1          __label__fake   
2  __label__partialError   
3          __label__fake   
4          __label__fake   
5          __label__fake   
6          __label__fake   
7          __label__fake   
8          __label__fake   
9  __label__partialError   

                                                                  title  \
0                              「洋蔥不能和魚、蝦、海帶、蜂蜜一起吃...會消化不良、產生結石、導致眼睛失明」？   
1  內容農場文章宣稱「大藥廠打死也不願說的秘密 原來抗癌可以那麼便宜...檸檬是一個神奇的植物，可殺死癌細胞。功效10,000倍強於化療」？   
2                       「家中長輩失智症是腦部血流堵塞造成。多吃藍莓，12天讓腦袋活過來，可抗腫瘤，促進腦部血流量」？   
3          「陳惠仁教授強調...熱苦瓜～只殺癌細胞！切２～３薄苦瓜片放在杯子裡， 加入熱水，會變鹼性水...苦瓜汁能調整高血壓」？   
4        「楊梅是皮蛇（帶狀皰疹）的尅星...10分鐘驗完血後，女醫生確定是得了帶狀皰疹...吃了兩天的楊梅後，居然神經不痛康復了」？   
5                                      「會浮在水面的麵是有加明礬的」、吃泡麵會得癌症、牙齒掉光、流產？   
6   圖片宣稱「感染肺炎的可怕，3到7天就開始發燒，感染5天後你就成為帶菌者，7到14天就會發作，20天多你的肺就會變成第4期，太可怕了」？   
7                                      「台大柯文哲醫師-生死的智慧，演講非常精采※空腹吃水果勝癌症」？   
8                    內容農場文章指出「全家篩檢女兒爸爸「都中大腸癌」！醫生警告香蕉和它（優酪乳）一起吃比「砒霜」還毒」？   
9                             組合圖片宣稱「隱翅蟲，在你身上時絕對不要打，牠身上有毒液，接觸到皮膚，就死定了」？   

                                       url  
0  https://tfc-taiwan.org.tw/articles/4025  
1  https://tfc-taiwan.org.tw/articles/4016  
2  https://tfc-taiwan.org.tw/articles/3975  
3  https://tfc-taiwan.org.tw/articles/3968  
4  https://tfc-taiwan.org.tw/articles/3953  
5  https://tfc-taiwan.org.tw/articles/3928  
6  https://tfc-taiwan.org.tw/articles/3923  
7  https://tfc-taiwan.org.tw/articles/3917  
8  https://tfc-taiwan.org.tw/articles/3916  
9  https://tfc-taiwan.org.tw/articles/3886

In [4]:
global content_list
content_list = []

def addC(t): 
    temp = []
    for p in t:
        temp.append(p.string)
    content_list.append(temp)

def getContent(url):
    req = request.Request(url)
    req.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.122 Safari/537.36")
    response = request.urlopen(req)
    data = response.read().decode("utf-8")

    #read data in html format
    html = BeautifulSoup(data,"html.parser")

    #set find area
    t = html.find_all("p")
    
    #add new content into list
    addC(t)
          
    time.sleep(3) 
    
for url in df['url']:
    getContent(url)
    #print(url)    

print(content_list)

[['Taiwan FactCheck Center', '經查：', '一、洋蔥為低草酸食物，正常攝取，不會有「洋蔥與魚同吃，造成人體對於魚的蛋白質無法吸收」狀況。', '二、洋蔥為低草酸食物，搭配魚、蝦或海帶一起煮，不會產生結石。', '三、洋蔥和蜂蜜都是天然食物，若均衡攝取，不會有失明風險。', '因此，傳言為「錯誤」訊息。', '背景', None, None, '查核', '爭議點一、傳言指稱「洋蔥和魚不能一起吃」，因為「洋蔥和魚同吃，洋蔥裡的草酸會分解、破壞魚中豐富的蛋白質，不容易被人體消化吸收」，是否屬實？', '（一）查核中心採訪衛生福利部臺北醫院營養科主任李佩霓。她指出，若攝取大量含草酸食物，食物中的草酸會干擾蛋白質的吸收，使得人體蛋白質消化吸收率變差。但洋蔥為低草酸食物，正常飲食狀況下並不會攝取大量洋蔥。', '例如一般民眾不會一餐吃八顆洋蔥、連吃一個月。所以不會有傳言所指的「洋蔥與魚同吃造成人體對於魚的蛋白質分解破壞，而不被吸收」的狀況。', '（二）查核中心採訪新光醫院營養師許為捷。她表示，草酸不會破壞蛋白質，而是與蛋白質一起結合沉澱，形成不溶性物質，從糞便排出。', '爭議點二、傳言指稱「洋蔥和蝦、海帶一起吃會形成草酸鈣形成結石」，是否屬實？', '（一）李佩霓主任表示，有許多因素影響結石的成因，一般健康狀態的人不會因為吃了特定食物就引起結石，例如「菠菜不能和豆腐同時食用」、與本傳言所指的「洋蔥和蝦」、「洋蔥和海帶」等說法，都是錯誤的觀念。', '正確的認知是，結石成分中有許多鈣，但是含鈣與含草酸的食物一起食用，並不是造成結石生成的主因。結石有可能是個人體質或遺傳等原因造成。', '李佩霓主任提醒，洋蔥，跟魚、蝦或搭配海帶一起煮並不產生結石，建議一般人把握均衡飲食、足量飲水的兩大原則即可。同時食用這些食物，並不會造成結石生成。', '李佩霓補充，對於具有結石體質的人，飲食要避免以下4種情況：', '1.水分攝取不足，排尿量過少。', '2.高鹽食物或鹽分攝取過多。', '3.蛋白質食物攝取過多。', '4.攝取高量維生素C（每日超過2000mg）', '（二）許為捷表示，「洋蔥和蝦」、「洋蔥和海帶」一起吃的確會形成少量草酸鈣，但草酸鈣會從糞便排出，並不會在人體器官內形成結石。', '爭議點三、傳言指稱「洋蔥和蜂蜜不能一起吃」，因為「一起吃對

In [5]:
#convert list to dataframe
df['content'] = content_list

print(df.head())

#output csv file
path = "C:\\Users\\user\Desktop\\newscrawler\\outputcsv\\csvdata\\"
df.to_csv(path + 'data2.csv', encoding='utf_8_sig', index=False, header=False)

tag  \
0          __label__fake   
1          __label__fake   
2  __label__partialError   
3          __label__fake   
4          __label__fake   

                                                                  title  \
0                              「洋蔥不能和魚、蝦、海帶、蜂蜜一起吃...會消化不良、產生結石、導致眼睛失明」？   
1  內容農場文章宣稱「大藥廠打死也不願說的秘密 原來抗癌可以那麼便宜...檸檬是一個神奇的植物，可殺死癌細胞。功效10,000倍強於化療」？   
2                       「家中長輩失智症是腦部血流堵塞造成。多吃藍莓，12天讓腦袋活過來，可抗腫瘤，促進腦部血流量」？   
3          「陳惠仁教授強調...熱苦瓜～只殺癌細胞！切２～３薄苦瓜片放在杯子裡， 加入熱水，會變鹼性水...苦瓜汁能調整高血壓」？   
4        「楊梅是皮蛇（帶狀皰疹）的尅星...10分鐘驗完血後，女醫生確定是得了帶狀皰疹...吃了兩天的楊梅後，居然神經不痛康復了」？   

                                       url  \
0  https://tfc-taiwan.org.tw/articles/4025   
1  https://tfc-taiwan.org.tw/articles/4016   
2  https://tfc-taiwan.org.tw/articles/3975   
3  https://tfc-taiwan.org.tw/articles/3968   
4  https://tfc-taiwan.org.tw/articles/3953   

                                                                                               content  
0  [Taiwan FactChec